# 注意：进入cmd输入以下命令，在谷歌浏览器里打开Tensorboard 
(tensorflow) D:\CQUPT\python\Jupyter Notebook\Deep Learning\Tensorflow>tensorboard --logdir=logs
[Tensorboard的安装及使用](https://blog.csdn.net/u012679707/article/details/79898530)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

# 每个批次的大小
batch_size = 100  # 以矩阵的形式放进去
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义命名空间
with tf.name_scope('input'):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')

with tf.name_scope('layer'):
    # 创建一个简单的神经网络
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='w')
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)

with tf.name_scope('loss'):
    # # 情形1：二次代价函数
    # loss = tf.reduce_mean(tf.square(y-prediction))
    # 情形2：对数似然代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

with tf.name_scope('train'):
    # 使用梯度下降算法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  # tf.argmax(prediction,1) 返回概率最大的位置（标签）
        # argmax返回一维张量中最大的值所在的位置
    with tf.name_scope('accuracy'):
        # 求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(1):  # 迭代
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter: ' + str(epoch) + ',Testing Accuracy: ' + str(acc))    